In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import *
import datetime
from dateutil.relativedelta import *
import pandas as pd
import numpy as np

In [ ]:
sc

# Cadastro

In [ ]:
cadastral = sc.textFile('CADASTRAL.txt')

In [ ]:
header1 = cadastral.first()
cadastral = cadastral.filter(lambda line: line != header1)

In [ ]:
cadastral_df = cadastral.map(lambda k: k.split("\t")).toDF(header1.split("\t"))
cadastral_df = cadastral_df.withColumn("DATA_CADASTRO", to_timestamp("DATA_CADASTRO", "yyyy/MM/dd HH:mm:ss"))
cadastral_df.show(5)

## Regra 1) Cadastrados a partir de 2017

In [ ]:
cadastral_df = cadastral_df.where(cadastral_df["DATA_CADASTRO"] >= "2017-01-01 00:00:00")


## Regra 2) Cadastrados até dezembro de 2017

In [ ]:
cadastral_df = cadastral_df.where(cadastral_df["DATA_CADASTRO"] < "2018-01-01 00:00:00")


## Cartão / Carnê (Linhas são reutilizadas para as duas bases)

Todas as linhas de código abaixo podem ser reaproveitadas paras as bases CARNE.TXT e CARTAO.TXT

In [ ]:
cartao = sc.textFile('CARNE.txt')

In [ ]:
header3 = cartao.first()
cartao = cartao.filter(lambda line: line != header3)

In [ ]:
cartao_df = cartao.map(lambda k: k.split("\t")).toDF(header3.split("\t"))
cartao_df = cartao_df.withColumn("DATA_VENCIMENTO", to_timestamp("DATA_VENCIMENTO", "yyyy/MM/dd HH:mm:ss"))
cartao_df = cartao_df.withColumn("DATA_PAGAMENTO", to_timestamp("DATA_PAGAMENTO", "yyyy/MM/dd HH:mm:ss"))
cartao_df.show(5)

## Inner Join - Cadastro+Cartão

In [ ]:
cadaux = cadastral_df.select('CODIGO_CLIENTE', 'DATA_CADASTRO')

In [ ]:
tcad = cadaux.alias('tcad')
tcart = cartao_df.alias('tcart')
cad_cart = tcad.join(tcart, tcad.CODIGO_CLIENTE == tcart.CODIGO_CLIENTE)
cad_cart.show(5)

Verifica se existe uma forma de rodar em PySpark.

In [ ]:
cad_cart = cad_cart.toPandas()

Adiciona 6 meses à data de cadastro para o limite de elegibilidade.

In [ ]:
cad_cart['DATA_LIMITE'] = cad_cart['DATA_CADASTRO'].apply(lambda x: x + relativedelta(months=+6))

Cria o flag que indica se a data de vencimento do contrato é anterior ao limite.

In [ ]:
cad_cart['FLAG_LIMITE'] = (cad_cart['DATA_VENCIMENTO'] <= cad_cart['DATA_LIMITE']).astype(int)

Cria o flag que indica se o pagamento foi atrasado ou não foi realizado.

In [ ]:
cad_cart['FLAG_ATRASO'] = ((cad_cart['DATA_PAGAMENTO'] > cad_cart['DATA_VENCIMENTO']) | (cad_cart['DATA_PAGAMENTO'].isnull())).astype(int)


Filtra os pagamentos elegíveis.

In [ ]:
cad_cart = cad_cart[cad_cart.FLAG_LIMITE > 0]

In [ ]:
len(cad_cart)

In [ ]:
cad_cart.head()

In [ ]:
cad_cart = cad_cart.iloc[:,1:]

Para cada cliente, verifica-se se o mesmo realizou todos os pagamentos em dia.

In [ ]:
selection = cad_cart.groupby('CODIGO_CLIENTE').FLAG_ATRASO.max()
selection[:10]

Separação entre "bons" e "maus" pagadores

In [ ]:
good = selection.loc[selection<1].index
bad = selection.loc[selection>0].index

In [ ]:
# Dataframe dos bons pagadores
good_df = pd.DataFrame()
good_df['CODIGO_CLIENTE'] = good
good_df['CLASS'] = np.zeros(len(good)).astype(int)

# Dataframe dos maus pagadores
bad_df = pd.DataFrame()
bad_df['CODIGO_CLIENTE'] = bad
bad_df['CLASS'] = np.ones(len(bad)).astype(int)

In [ ]:
good_df.head()

In [ ]:
bad_df.head()

União dos dois dataframes criados acima. <b>P.S: Lembrar de mudar o nome para "classificados_carne" ou "classificados_cartao", dependendo da base escolhida.</b>

In [ ]:
classificados_carne = good_df.append(bad_df)

# Junção dos classificados

In [ ]:
classificados_cartao.head()

In [ ]:
classificados_carne.head()

In [ ]:
classificados = classificados_carne.append(classificados_cartao)

In [ ]:
final = classificados.groupby("CODIGO_CLIENTE").CLASS.max()

In [ ]:
tabelao = pd.DataFrame()

In [ ]:
tabelao['CODIGO_CLIENTE'] = final.index
tabelao['CLASS'] = final.values

In [ ]:
tabelao.groupby("CLASS").size()

In [ ]:
tabelao.to_csv('classified.csv',index=False)

# Join: Cadastral / Classificados

In [ ]:
class_rdd = sc.textFile('classified.csv')

In [ ]:
header_class = class_rdd.first()
class_rdd = class_rdd.filter(lambda line: line!=header_class)
class_df = class_rdd.map(lambda k: k.split(",")).toDF(header_class.split(","))

In [ ]:
class_df.count()

In [ ]:
tcad = cadastral_df.alias('tcad')
tclass = class_df.alias('tclass')
final_cadastral = tcad.join(tclass, tcad.CODIGO_CLIENTE == tclass.CODIGO_CLIENTE, 'inner')
final_cadastral.show(5)

# -------------------------------------------
# Tabelão
# -------------------------------------------

In [ ]:
FINAL = final_cadastral.toPandas()

In [ ]:
FINAL.to_csv('tabelao.csv',index=False)